* multiple random iteration around optimal value
    - evidence : escape the trap
    - normal distribution
* C-shunted qubit : profile aping

In [1]:
from multiprocess import Pool
#from multiprocessing import Pool
from DiSuQ.Torch.optimization import uniformParameters,initializationParallelism,truncNormalParameters,initializationSequential
from DiSuQ.Torch import models
from DiSuQ.Torch.optimization import OrderingOptimization,PolynomialOptimization
from DiSuQ.Torch.optimization import lossTransition
from torch import tensor, float32 as float
from numpy import arange,linspace
from DiSuQ.utils import plotCompare
import pickle
from torch import set_num_threads
set_num_threads(32)

In [2]:
with open('./target_fluxqubit.p', 'rb') as content: target_info = pickle.load(content)
target_spectrum = target_info['spectrum']
E10 = target_spectrum[:,1] - target_spectrum[:,0]
E20 = target_spectrum[:,2] - target_spectrum[:,0]
target = {'E10':E10[[0,20,-1]],'E20':E20[[0,20,-1]]}

In [3]:
basis = {'O':[4],'J':[8,8],'I':[]}; rep = 'K'
circuit = models.shuntedQubit(basis,sparse=False)

In [4]:
flux_range = linspace(0,1,3,endpoint=True)

In [5]:
flux_profile = [{'I':flux} for flux in flux_range]

In [6]:
lossObjective = lossTransition(tensor(target['E10'],dtype=float),tensor(target['E20'],dtype=float))

In [7]:
optim = OrderingOptimization(circuit,representation=rep)

#### parameter space

In [8]:
circuit.modeDistribution()

(1, 0, 2)

In [9]:
n_init = 20; var = 30
parameters = truncNormalParameters(circuit,n_init,var)

## Multi Initialization

In [10]:
iterations = 50
lr = .0001

In [11]:
target

{'E10': array([18.48131364,  5.04390444, 18.48131364]),
 'E20': array([38.41531441, 11.10033571, 38.41531441])}

In [12]:
#with Pool(5) as multi:
#    Search = multi.map(initializationParallelism(optim,lossObjective,flux_profile,iterations=iterations,lr=lr),parameters)

In [13]:
Search = initializationSequential(parameters,optim,lossObjective,flux_profile,iterations=iterations,lr=lr)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [14]:
pickle.dump(Search,open( "search-C-shunt.p", "wb" ))

### optimization analysis

In [15]:
Losses = dict()
for index,(parameter,(dLogs,dParams,dCircuit)) in enumerate(zip(parameters,Search)):
    loss = dLogs['loss'].to_numpy()
    if len(loss) < 1:
        continue
    #if loss[0]-loss[-1] > 0:       
    Losses[str(index)] = loss

In [18]:
plotCompare(arange(iterations),Losses,'C-shunted Profiling','iterations','loss')